<h1> Материалы курса "Анализ данных в инфромационной безопасности" </h1>

<h2> Домашнее задание к лекции "Анализ данных" </h2>

<p> Андрей Скороходов, группа UDV</p>
Бахарев С.М., УрФУ

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import missingno as msno

## 1-2. Загрузка данных

1.	Загрузите исходный датасет Практика/2.1/data_example.csv

pd.read_csv(data_example.csv)

2.	Сделайте необходимые преобразования при загрузке датасета, чтобы данные выглядели корректно.
https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html, корректно обработайте разделители, NA значения, кодировку, исключите ненужные строки, определите заголовок, при необходимости игнорируйте ошибки в кодировке или подправьте исходный файл.

In [ ]:
file_path = 'data_example.csv'
data = pd.read_csv(file_path,
                   sep=",",
                   header=[0,1,2,3],
                   skiprows=1,
                   na_values=['Missing','nan','Offln'],
                   encoding_errors='ignore'
                  )
data

## 3. Очистка заголовков

3.	Очистите заголовки колонок от ненужной информации, оставьте только имя устройства и название параметра

## 4. Информация о датасете 

4.	Выведите в консоль информацию о датасете: его размер, название колонок, их типы данных
df.info()

## 5. Работа с индексом времени
Преобразуйте колонку со временем в формат datetime, сделайте ее индексом и отсортируйте (даже если датасет уже отсортирован)
pd.to_datetime(), df.index = df[<колонка>], df.sort_index()

In [ ]:
data['time'] = pd.to_datetime(data['time'], format=' %H:%M:%S %d/%m/%Y')
data.index = data.pop('time')
data = data.sort_index()
data

## 6. Пропущенные значения

Посчитайте количество пропусков в каждой колонке и визуализируйте их
df.isna().sum(), для визуализации можно использовать модуль missingno и метод matrix(df)


Пропущенные значения заполните их предыдущими значениями. 

## 7. Типы данных 
Как оптимизировать типы данных в колонках? Насколько меньше станет размер датафрейма после оптимизации типов?
Приведите к типу int колонки с целыми значениями, используйте df.astype(). 
Приведите к типу category колонки с категориальными признаками (признаки, которые принимают ограниченное количество значений, например до 10). 
Сравните замеры получившихся датасетов, оригинальный датасет не перезаписывайте.


Для определения целочисленных колонок воспользуемся знаниями предметной области:
- EKC состояние - это состояние устройства, точно целочисленное, возможно категориальное
- признаки с префиксом Cap или Capacity - это производительность устройств, тоже целочисленное

## 8. Работа с непрерывными признаками
Дайте табличное и графическое статистическое описание признакам, содержащим параметр «Sair» для устройств 21CT, 22CT,…, 30CT.
df.describe(), sns.boxplot(). 


In [ ]:
col_Sair = [...]
col_Sair

In [ ]:
plt.figure(figsize=(20, 6))
boxplot = sns.boxplot(data=data[col_Sair])
boxplot.set(title='Boxplot')
boxplot.set_xticklabels(boxplot.get_xticklabels(), rotation=45)
plt.show()

## 9. Ресэмплирование
Ресемплируйте датасет по медианному значению за 4 минуты, отобразите на линейном графике значение признаков col_Sair. График должен быть читабельным, иметь заголовок и легенду, можете выбрать другое значение ресэмплирования, чтобы график лучше читался.
Используйте любые модули (df.plot(), sns, plt, plotly). Ресемплированный датасет сохранять не нужно, просто отобразить.

In [ ]:
data[col_Sair].resample('...').median().plot(figsize=(20,5), title='Ресэмплированные значения признаков с Sair')

## 10. Сглаживание
Постройте для 2-3 признаков из col_Sair сглаженный график поведения признаков во времени. Ширину окна выберете такую, чтобы был виден тренд.

In [ ]:
data[col_Sair[:3]].rolling(...).mean().plot(figsize=(20,5), title='Сглаженные значения признаков с Sair')

## 11. Визуализация разницы между ресэмплингом и сглаживанием

Для корректного отображения нам нужно привязываться не целочисленным индексам датафрейма, а к временным меткам, для этого нужно использовать df.loc[время от: время до].

Мы выберем одинаковые промежутки: 12 минут для ресэмплинга и 6 сэмплов для сглаживания (у нас сэмплы идут с периодом 2 минуты)

In [ ]:
time_from = '2021-03-01 06:00'
time_till = '2021-03-01 18:00'

plt.title('Разница между сглаживанием и ресэмплингом')
data[col_Sair[0]].loc[time_from:time_till].plot(figsize=(20,5), label='original', alpha=0.5)
data[col_Sair[0]].resample('12T').mean().loc[time_from:time_till].plot(figsize=(20,5), label='resample 12 min')
data[col_Sair[0]].rolling(6).mean().loc[time_from:time_till].plot(figsize=(20,5), label='rolling 6')
plt.legend()
plt.show()

## 12. Гистограммы и столбчатые диаграммы
Отобразите гистограмму по количеству одновременно включенных устройств. Признак состояния представлены в виде «<устройство>: --- EKC состояние», если признак принимает значение 0, то считаем устройство включенным.

In [ ]:
EKCstate_Map = data[[col for col in data.columns if '---' in col]].applymap(lambda x: 1 if x==0 else 0)
EKCstate_Map.sum(axis=1).value_counts().sort_index().plot.bar(figsize=(5,5), title = 'Количество одновременно включенных горок')

## 13. Subplots

Постройте для нескольких признаков состояния устройств гистограммы, которые покажут распределение этих устройств по состояниям.
 
Используйте любой plt.bar или sns.barplot() и plt.subplots()

## 14. Матрица корреляции
Постройте матрицу корреляции для признаков col_Sair
Получение таблицы с корреляцией: df.corr(), отрисовку удобно делать через sns.heatmap()

## 15. Проверьте гипотезу о том, что тренд температуры u09 S5 в устройствах 12СТ и 13СТ зависит от состояния устройства. Свои выводы подтвердите визуализацией или статистиками.

In [ ]:
cols = [...]

df = data[cols].copy()
df.columns = ['Temperature', 'State']
df

In [ ]:
# Ваш код

**Выводы:** ...


## 16. Прогнозирование временных рядов.

1. В датасете data оставьте один признак "21CT G GSR +2/+4:   u69 Sair Темп ".
2. Сделайте ресэмплирование по медианному значению за 60 минут. Постройте график.
3. Сделайте сглаживание с окном 10. Постройте график.



In [2]:
# Ваш код

### 16.1 Автокорреляция и частичная автокорреляция

Посторойте графики автокорреляционной функции и частичной автокорреляции. Можно использовать pd.plotting.autocorrelation_plot(data) и statsmodels.api.graphics.tsa.plot_pacf(data).
Сделайте выводы.

In [ ]:
# Ваш код

### 16.2 Прогонозирование с помощью ARIMA
 
ARIMA - авторегрессионное интегрированное скользящее среднее. ARIMA позволяет моделировать данные, не являющиеся стационарными. ARIMA включает три параметра: параметр авторегрессии (р), параметр скользящего среднего (q) и параметр интегрирования (d). 


https://education.yandex.ru/handbook/ml/article/modeli-vida-arima

1. На основе анализа автокорреляционной функции и частичной автокорреляции подберите параметры (гиперпараметры) модели p, q и d.
2. Разбейте датасет на обучающую и тестовую выборки.
3. Сделайте прогноз и сравните с тестовой выборкой.
4. Оцените точность прогноза, используя метрики rmse, mae.
5. На одном графике постройте предсказанную и истинную (test) температуры.
6. Сделайте выводы.

In [ ]:
from statsmodels.tsa.arima.model import ARIMA
from math import sqrt
from sklearn.metrics import mean_squared_error
from sklearn.metrics import ...

In [ ]:
# Разбейте датасет на обучающую и тестовую выборки
X = data.values
size = int(len(X) * 0.80)
train, test = X[0:size], X[size:len(X)]

history = [x for x in train]
predictions = list()

# Пошаговая валидация
for t in range(len(test)):
	model = ARIMA(history, order=(...))
    # Ваш код
    
    
    
# Оценка модели
rmse = sqrt(mean_squared_error(test, predictions))
mae = ...
print('Test RMSE: %.3f' % rmse)
print('Test MAE: %.3f' % mae)

# Plotting
plt.plot(test)
plt.plot(predictions, color='red')
plt.show()